In [10]:
# Setup the Jupyter version of Dash
from jupyter_dash import JupyterDash

# Configure the necessary Python module imports for dashboard components
import dash_leaflet as dl
from dash import dcc, html
import plotly.express as px
from dash import dash_table
from dash.dependencies import Input, Output, State
import base64
JupyterDash.infer_jupyter_proxy_config()

# Configure OS routines
import os

# Configure the plotting routines
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


#### FIX ME #####
# change animal_shelter and AnimalShelter to match your CRUD Python module file name and class name
from CRUD_Python_Module import CRUD as AnimalShelter

###########################
# Data Manipulation / Model
###########################
# FIX ME update with your username and password and CRUD Python module name

username = "aacuser"
password = "NewPassword123"

# Connect to database via CRUD Module
db = AnimalShelter(username=username, password=password)

# class read method must support return of list object and accept projection json input
# sending the read method an empty document requests all documents be returned
df = pd.DataFrame.from_records(db.find_documents("animals", {}))

# MongoDB v5+ is going to return the '_id' column and that is going to have an 
# invlaid object type of 'ObjectID' - which will cause the data_table to crash - so we remove
# it in the dataframe here.
df.drop(columns=['_id'], inplace=True)

## Debug
# print(len(df.to_dict(orient='records')))
# print(df.columns)


#########################
# Dashboard Layout / View
#########################
app = JupyterDash(__name__)

# FIX ME Add in Grazioso Salvare’s logo
image_filename = 'Grazioso Salvare Logo.png'  # Replace with your own image if needed
encoded_image = base64.b64encode(open(image_filename, 'rb').read())

# FIX ME Place the HTML image tag in the line below into the app.layout code according to your design
# FIX ME Also remember to include a unique identifier such as your name or date
app.layout = html.Div([
    html.Center(html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode()), style={'width': '200px'})),
    html.Center(html.B(html.H1('CS-340 Dashboard - Thalia Stafford'))),
    html.Hr(),

    # FIXME Add in code for the interactive filtering options.
    html.Div([
        dcc.RadioItems(
            id='filter-type',
            options=[
                {'label': 'Water Rescue', 'value': 'Water Rescue'},
                {'label': 'Mountain/Wilderness Rescue', 'value': 'Mountain or Wilderness Rescue'},
                {'label': 'Disaster/Individual Tracking', 'value': 'Disaster or Individual Tracking'},
                {'label': 'Reset', 'value': 'Reset'}
            ],
            value='Reset',
            inline=True
        )
    ]),

    html.Hr(),
    dash_table.DataTable(
        id='datatable-id',
        columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns],
        data=df.to_dict('records'),

        # FIXME: Set up the features for your interactive data table
        editable=False,
        filter_action="native",
        sort_action="native",
        sort_mode="multi",
        column_selectable="single",
        row_selectable="single",
        row_deletable=False,
        page_action='native',
        page_current=0,
        page_size=10,
        style_table={'overflowX': 'auto'},
        style_cell={
            'minWidth': '100px', 'width': '150px', 'maxWidth': '200px',
            'whiteSpace': 'normal',
            'textAlign': 'left'
        }
    ),
    html.Br(),
    html.Hr(),

    # This sets up the dashboard so that your chart and your geolocation chart are side-by-side
    html.Div(className='row', style={'display': 'flex'}, children=[
        html.Div(id='graph-id', className='col s12 m6'),
        html.Div(id='map-id', className='col s12 m6')
    ])
])


#############################################
# Interaction Between Components / Controller
#############################################


@app.callback(Output('datatable-id', 'data'),
              [Input('filter-type', 'value')])
def update_dashboard(filter_type):
    # FIX ME Add code to filter interactive data table with MongoDB queries
    if filter_type == 'Water Rescue':
        data = pd.DataFrame.from_records(db.find_documents("animals", {"animal_type": "Dog", "breed": {"$in": ["Labrador Retriever Mix", "Chesapeake Bay Retriever", "Newfoundland"]}}))
    elif filter_type == 'Mountain or Wilderness Rescue':
        data = pd.DataFrame.from_records(db.find_documents("animals", {"animal_type": "Dog", "breed": {"$in": ["German Shepherd", "Alaskan Malamute", "Old English Sheepdog", "Siberian Husky", "Rottweiler"]}}))
    elif filter_type == 'Disaster or Individual Tracking':
        data = pd.DataFrame.from_records(db.find_documents("animals", {"animal_type": "Dog", "breed": {"$in": ["Doberman Pinscher", "German Shepherd", "Golden Retriever", "Bloodhound", "Rottweiler"]}}))
    else:
        data = pd.DataFrame.from_records(db.find_documents("animals", {}))

    if '_id' in data.columns:
        data.drop(columns=['_id'], inplace=True)
    return data.to_dict('records')


# Display the breeds of animal based on quantity represented in the data table
@app.callback(
    Output('graph-id', "children"),
    [Input('datatable-id', "derived_virtual_data")])
def update_graphs(viewData):
    ### FIX ME ####
    dff = pd.DataFrame.from_dict(viewData)
    fig = px.pie(dff, names='breed', title='Preferred Dog Breeds')
    return [dcc.Graph(figure=fig)]


# This callback will highlight a cell on the data table when the user selects it
@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)
def update_styles(selected_columns):
    if not selected_columns:
        return []  # no columns selected yet, return empty styling
    return [{
        'if': {'column_id': i},
        'background_color': '#D2F3FF'
    } for i in selected_columns]


# This callback will update the geo-location chart for the selected data entry
@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_virtual_data"),
     Input('datatable-id', "derived_virtual_selected_rows")])
def update_map(viewData, index):
    if not viewData:
        return
    if not index:
        index = [0]

    dff = pd.DataFrame.from_dict(viewData)
    row = index[0]

    return [
        dl.Map(style={'width': '1000px', 'height': '500px'}, center=[dff.iloc[row]['location_lat'], dff.iloc[row]['location_long']], zoom=10, children=[
            dl.TileLayer(id="base-layer-id"),
            dl.Marker(position=[dff.iloc[row]['location_lat'], dff.iloc[row]['location_long']], children=[
                dl.Tooltip(dff.iloc[row]['breed']),
                dl.Popup([
                    html.H4("Animal Details"),
                    html.P(f"Name: {dff.iloc[row]['name']}"),
                    html.P(f"Breed: {dff.iloc[row]['breed']}"),
                    html.P(f"Type: {dff.iloc[row]['animal_type']}")
                ])
            ])
        ])
    ]


# Run app
app.run_server()


Dash app running on https://bronzebeast-taxibarbara-3000.codio.io/proxy/8050/
